In [14]:
import pandas as pd

In [5]:
df.describe

<bound method NDFrame.describe of           fdc_id                               brand_owner  \
0        1105904  Richardson Oilseed Products (US) Limited   
1        1105905                     CAMPBELL SOUP COMPANY   
2        1105906                     CAMPBELL SOUP COMPANY   
3        1105907                     CAMPBELL SOUP COMPANY   
4        1105908                     CAMPBELL SOUP COMPANY   
...          ...                                       ...   
1958973  2688917                Incobrasa Industries, Ltd.   
1958974  2688918                          Tyson Foods Inc.   
1958975  2688919                          Tyson Foods Inc.   
1958976  2688920                          Tyson Foods Inc.   
1958977  2688921                    Clemens Food Group LLC   

                 brand_name subbrand_name        gtin_upc  \
0                       NaN           NaN     27000612323   
1                       NaN           NaN     51000198808   
2                       NaN           

In [2]:
import psycopg2
import ipywidgets as widgets
from IPython.display import display, clear_output

class PostgresApp:
    def __init__(self):
        # Database connection attributes
        self.connection = None
        self.cursor = None

        # Interface widgets
        self.host_entry = widgets.Text(description="Host:")
        self.database_entry = widgets.Text(description="Database:")
        self.user_entry = widgets.Text(description="Username:")
        self.password_entry = widgets.Password(description="Password:")
        
        self.connect_button = widgets.Button(description="Connect")
        self.connect_button.on_click(self.connect_to_database)

        self.query_text = widgets.Textarea(description="SQL Query:", layout=widgets.Layout(width="80%", height="100px"))
        self.run_button = widgets.Button(description="Run Query")
        self.run_button.on_click(self.run_query)

        # Exit button with padding added
        self.close_button = widgets.Button(description="Exit", layout=widgets.Layout(margin='10px 0px 0px 20px'))
        self.close_button.on_click(self.close_application)

        self.result_output = widgets.Output(layout=widgets.Layout(width="80%", height="200px"))

        # Layout display with Run and Exit buttons side by side but spaced
        self.main_layout = widgets.VBox([
            widgets.Label("Database Connection Details"),
            self.host_entry, self.database_entry, self.user_entry, self.password_entry,
            self.connect_button,
            widgets.Label("SQL Query Execution"),
            self.query_text, 
            widgets.HBox([self.run_button, self.close_button]),  # Positioned Run and Exit together with spacing
            self.result_output
        ])
        display(self.main_layout)

    def connect_to_database(self, _=None):
        """Connect to the PostgreSQL database with provided credentials."""
        try:
            self.connection = psycopg2.connect(
                host=self.host_entry.value,
                database=self.database_entry.value,
                user=self.user_entry.value,
                password=self.password_entry.value
            )
            self.cursor = self.connection.cursor()
            with self.result_output:
                clear_output()
                print("Connected to the database successfully.")
        except (Exception, psycopg2.DatabaseError) as error:
            with self.result_output:
                clear_output()
                print(f"Failed to connect to the database: {error}")

    def run_query(self, _=None):
        """Run SQL query, display query with results or confirmation."""
        if not self.connection:
            with self.result_output:
                clear_output()
                print("Please connect to the database first.")
            return

        query = self.query_text.value.strip()
        if not query:
            with self.result_output:
                clear_output()
                print("Please enter a SQL query to execute.")
            return

        try:
            self.cursor.execute(query)
            with self.result_output:
                clear_output()
                print(f"Executed Query:\n{query}\n")  # Display the query

                if query.lower().startswith("select"):
                    rows = self.cursor.fetchall()
                    self.display_results(rows)
                else:
                    self.connection.commit()
                    print("Query executed successfully.")

            # Clear the query input area for a new query
            self.query_text.value = ""

        except (Exception, psycopg2.DatabaseError) as error:
            # Rollback to clear the transaction state
            self.connection.rollback()
            with self.result_output:
                clear_output()
                print(f"Error while executing query: {error}")

    def display_results(self, rows):
        """Display query results below the query."""
        with self.result_output:
            for row in rows:
                print(row)

    def close_application(self, _=None):
        """Close the application by clearing UI components and closing the connection."""
        self.close_connection()
        self.main_layout.close()  # Hides all widgets
        with self.result_output:
            clear_output()
            print("Application closed. To restart, re-run the notebook cell.")

    def close_connection(self):
        """Close the database connection."""
        if self.connection:
            self.cursor.close()
            self.connection.close()
            print("PostgreSQL connection is closed.")

# Initialize and display the app
app = PostgresApp()


PostgreSQL connection is closed.
